In [2]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy

In [10]:
!ls ../../ppc/scripts

autotune_12000.sh
autotune_eval_10000.sh
autotune_eval_12000_seed.sh
autotune_eval_12000.sh
autotune_eval_8000.sh
autotune_eval_zifa_A.sh
autotune_eval_zifa_B.sh
autotune_eval_zifa_C_new.sh
autotune_eval_zifa_C.sh
autotune_eval_zifa_D_new.sh
autotune_eval_zifa_D.sh
autotune_eval_zifa_F_new.sh
autotune_eval_zifa_F.sh
autotune_eval_zifa_F_sos10.sh
autotune_eval_zifa_G_new.sh
autotune_eval_zifa_G.sh
autotune.sh
autotune_zi_12000.sh
autotune_zifa_new.sh
autotune_zifa.sh
autotune_zi.sh
aws_sos
brain_small
brain_small_nb_1200_50_results.json
brain_small_old
brain_small_zinb_1200_50_results.json
brain_small.zip
catchup_new.txt
catchup.txt
check_hemato.py
cortex
cortex_bis
cortex_nb_1200_50_results.json
cortex_old
cortex_zinb_1200_50_results.json
cortex.zip
data
hemato
hemato_nb_1200_50_results.json
hemato_old
hemato_zinb_1200_50_results.json
hemato_zinb_1200_50_results_new.json
hemato.zip
klein_dataset
klein_dataset_nb_1200_50_results.json
klein_dataset_zinb_1200_50_results.json
log_poisson_n

In [3]:


data_path = '../../ppc/scripts'
dataset_names = {'log_poisson_zifa_dataset_12000_0.08_{}'.format(zifa_lambda):
                 '$\lambda = {}$'.format(str(zifa_lambda).rstrip('0').rstrip('.'))
                 for zifa_lambda in [0., 1e-3, 1e-2, 1e-1, 1.0, 1e1, 1e2, 1e3]}

dataset_names = {'cortex': 'Cortex',
                 'hemato': 'Hemato',
                 'brain_small': 'Brain Small',
                 'pbmc': 'PBMC',
                 'retina': 'Retina'}

data_files = {}
data_names = {}
for dataset_name in dataset_names:
    data_files[dataset_name] = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
    data_names[dataset_name] = []
    for data_file in data_files[dataset_name]:
        data_name = data_file.split('/')[-1].split('_')[0]
        data_names[dataset_name].append(data_name)

for dataset_name in list(dataset_names.keys()):
    if len(data_names[dataset_name]) != 2:
        del dataset_names[dataset_name]
        del data_files[dataset_name]
        del data_names[dataset_name]

In [4]:
dataset_names

{'cortex': 'Cortex',
 'hemato': 'Hemato',
 'brain_small': 'Brain Small',
 'pbmc': 'PBMC',
 'retina': 'Retina'}

In [5]:
data_names

{'cortex': ['nb', 'zinb'],
 'hemato': ['nb', 'zinb'],
 'brain_small': ['nb', 'zinb'],
 'pbmc': ['nb', 'zinb'],
 'retina': ['nb', 'zinb']}

In [6]:
df = {}
for dataset_name in dataset_names:
    dfs = []
    for data_name, f in zip(data_names[dataset_name], data_files[dataset_name]):
        if data_name != 'zifa_full':

            #my_df = pd.read_csv(f, sep='\t')
            my_df = pd.read_pickle(f)
            my_df.loc[:, 'model'] = data_name
            dfs.append(my_df)
            print(data_name, f)
    #         display(my_df)
    df[dataset_name] = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/cortex/nb_cortex.p
zinb ../../ppc/scripts/cortex/zinb_cortex.p
nb ../../ppc/scripts/hemato/nb_hemato.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


zinb ../../ppc/scripts/hemato/zinb_hemato.p
nb ../../ppc/scripts/brain_small/nb_brain_small.p
zinb ../../ppc/scripts/brain_small/zinb_brain_small.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


nb ../../ppc/scripts/pbmc/nb_pbmc.p
zinb ../../ppc/scripts/pbmc/zinb_pbmc.p
nb ../../ppc/scripts/retina/nb_retina.p
zinb ../../ppc/scripts/retina/zinb_retina.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [7]:
from statsmodels.stats.weightstats import ttest_ind

metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']

#h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
h1_hypothesis = ['greater', 'greater', 'greater', 'greater', 'greater']

h1_hypothesis_bis = ['smaller', 'smaller', 'smaller', 'smaller', 'smaller']

df_summary = {}

for dataset_name in dataset_names:

    df_nb = df[dataset_name].loc[df[dataset_name].model=='nb', metrics]
    df_zinb = df[dataset_name].loc[df[dataset_name].model=='zinb', metrics]

    def get_pvals(gby, other_df):
        my_df = gby[metrics]
        print(gby)
        assert my_df.shape[1] == len(metrics)
        pvals = []
        for idx, alternative in enumerate(h1_hypothesis):
            assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
            #_, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
            _, pval = scipy.stats.mannwhitneyu(other_df.values[:, idx], my_df.values[:, idx], use_continuity=True, alternative=alternative)
            pvals.append(pval)
        return np.array(pvals)

    pvals_against_zinb = df[dataset_name].groupby('model').apply(get_pvals, other_df=df_zinb)
    pvals_against_zinb = (pvals_against_zinb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    pvals_against_nb = df[dataset_name].groupby('model').apply(get_pvals, other_df=df_nb)
    pvals_against_nb = (pvals_against_nb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    def get_summary(gby):
        res = {}
        res['mean'] = gby.mean()
        res['std'] = gby.std()
        res['pvals_against_nb'] = pvals_against_nb[gby.name]
        res['pvals_against_zi'] = pvals_against_zinb[gby.name]
        return pd.DataFrame(res).T

    # df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
    #                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
    df_summary[dataset_name] = df[dataset_name].groupby('model')[metrics].apply(get_summary)


    df_summary[dataset_name] = df_summary[dataset_name].stack().unstack(1).sort_index(level=1).swaplevel()

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

                 gene_dropout_gene_dropout_logits_all  \
0   [-5.627172, 1.3879467, -2.6609066, -5.637821, ...   
1   [-5.7150784, 1.7188218, -2.165279, -5.139339, ...   
2   [-5.7138376, 1.5640235, -2.430982, -5.268702, ...   
3   [-5.841869, 1.7730346, -2.520898, -5.451619, 2...   
4   [-5.6847525, 1.7340777, -2.582346, -5.192821, ...   
5   [-5.368562, 1.8106368, -2.1797652, -5.024699, ...   
6   [-5.7406173, 1.9538027, -2.4427094, -5.0252523...   
7   [-5.8346257, 1.9049379, -2.5384858, -4.6490088...   
8   [-5.749211, 1.6281065, -2.7558892, -5.042511, ...   
9   [-6.067636, 1.7508578, -2.4506702, -5.7717786,...   
10  [-5.604969, 1.7686548, -2.374416, -4.9849753, ...   
11  [-5.648022, 1.6632959, -2.4557078, -5.3438115,...   
12  [-5.764522, 1.2910242, -2.5283523, -5.323476, ...   
13  [-6.058365, 2.002632, -2.3342469, -4.8524876, ...   
14  [-5.571825, 1.8598652, -2.2860136, -5.396693, ...   
15  [-6.525749, 1.670227, -2.5396836, -5.4195294, ...   
16  [-6.0797915, 1.7514023, -2.

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

...

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

                 gene_dropout_gene_dropout_logits_all  \
0   [-7.6816444, -7.7537494, -5.3450456, -0.747957...   
1   [-7.46807, -7.5063868, -5.2153835, -0.49798557...   
2   [-7.588729, -7.689326, -5.2362356, -0.75076675...   
3   [-7.602187, -7.592971, -5.105736, -0.47086284,...   
4   [-7.9761705, -8.021966, -5.573136, -0.6985248,...   
5   [-7.6286373, -7.685562, -5.2253165, -0.5159529...   
6   [-7.709907, -7.747672, -5.348109, -0.533335, -...   
7   [-7.5516477, -7.577682, -5.3891273, -0.6201684...   
8   [-7.610735, -7.6429725, -5.2061095, -0.4904832...   
9   [-7.894266, -7.9649053, -5.351781, -0.6663839,...   
10  [-7.6946115, -7.8363404, -5.245485, -0.8442003...   
11  [-7.6507025, -7.700857, -5.3700066, -0.5806205...   
12  [-7.5709376, -7.5898347, -5.2473207, -0.539362...   
13  [-7.7449803, -7.7967725, -5.4512076, -0.707930...   
14  [-7.441109, -7.5703564, -5.0991297, -0.5631683...   
15  [-7.5355825, -7.606762, -5.27018, -0.5546697, ...   
16  [-7.626802, -7.665482, -5.2

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

                 gene_dropout_gene_dropout_logits_all  \
0   [-7.6816444, -7.7537494, -5.3450456, -0.747957...   
1   [-7.46807, -7.5063868, -5.2153835, -0.49798557...   
2   [-7.588729, -7.689326, -5.2362356, -0.75076675...   
3   [-7.602187, -7.592971, -5.105736, -0.47086284,...   
4   [-7.9761705, -8.021966, -5.573136, -0.6985248,...   
5   [-7.6286373, -7.685562, -5.2253165, -0.5159529...   
6   [-7.709907, -7.747672, -5.348109, -0.533335, -...   
7   [-7.5516477, -7.577682, -5.3891273, -0.6201684...   
8   [-7.610735, -7.6429725, -5.2061095, -0.4904832...   
9   [-7.894266, -7.9649053, -5.351781, -0.6663839,...   
10  [-7.6946115, -7.8363404, -5.245485, -0.8442003...   
11  [-7.6507025, -7.700857, -5.3700066, -0.5806205...   
12  [-7.5709376, -7.5898347, -5.2473207, -0.539362...   
13  [-7.7449803, -7.7967725, -5.4512076, -0.707930...   
14  [-7.441109, -7.5703564, -5.0991297, -0.5631683...   
15  [-7.5355825, -7.606762, -5.27018, -0.5546697, ...   
16  [-7.626802, -7.665482, -5.2

# New export Latex

In [8]:
def boldizer(row):
    to_bold = (row.pvals_against_nb <= 5e-2) | (row.pvals_against_zi <= 5e-2)
#     print(to_bold)
    if to_bold:
        row.latex = '\mathbf{' + row.latex + '}'
    return row

In [9]:
df_summary

{'cortex':                                                  mean        std  \
                                    model                           
 imputation_median_imputation_score nb        2.317937   0.153119   
                                    zinb      2.458491   0.290626   
 ll_ll                              nb     1276.231442  18.531582   
                                    zinb   1285.789172  27.041058   
 t_cv_ks_stat                       nb        0.212760   0.033852   
                                    zinb      0.246882   0.038721   
 t_dropout_ks_stat                  nb        0.087240   0.039732   
                                    zinb      0.049462   0.018110   
 t_ratio_ks_stat                    nb        0.108530   0.055928   
                                    zinb      0.107384   0.069493   
 
                                           pvals_against_nb  pvals_against_zi  
                                    model                                      


In [10]:
n_dec = 3

work_df = {}

for dataset_name in dataset_names:

    work_df[dataset_name] = (df_summary[dataset_name]
               .reset_index(level=-1)
               .reset_index()
               .assign(latex=lambda x: x['mean'].round(n_dec).astype(str) + ' \pm ' + x['std'].round(n_dec).astype(str))
               .apply(boldizer, axis=1)
               .assign(Metric=lambda x: 
                       x.loc[:, 'index'].replace({
                           'imputation_median_imputation_score': 'Imputation Score',
                           'll_ll': 'Log Likelihood',
                           't_cv_ks_stat': 'PPC (Coefficient of Variation)',
                           't_dropout_ks_stat': 'PPC (Dropout)',
                           't_ratio_ks_stat': 'PPC (Ratio of zeros)',
                           'model': 'Model'
               }),
                       latex=lambda x: '$' + x.latex + '$'
                      )
              )

    work_df[dataset_name] = work_df[dataset_name].rename(columns={'model': 'Model', 'index': 'metric'})
    work_df[dataset_name]['Model'] = work_df[dataset_name]['Model'].str.upper()
    #work_df[dataset_name]['Dataset'] = dataset_names[dataset_name]
    

In [11]:
work_df.keys()

dict_keys(['cortex', 'hemato', 'brain_small', 'pbmc', 'retina'])

In [12]:
dataset_names

{'cortex': 'Cortex',
 'hemato': 'Hemato',
 'brain_small': 'Brain Small',
 'pbmc': 'PBMC',
 'retina': 'Retina'}

In [13]:
pivot = {}
for dataset_name in dataset_names:
    pivot[dataset_name] = pd.pivot(work_df[dataset_name], index='Metric', values='latex', columns='Model')
    pivot[dataset_name]['Dataset'] = dataset_names[dataset_name]
    pivot[dataset_name].reset_index(inplace=True)
    pivot[dataset_name].set_index(['Dataset', 'Metric'], inplace=True)
pivot = pd.concat(list(pivot.values()), axis=0)

In [14]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 25 entries, (Cortex, Imputation Score) to (Retina, PPC (Ratio of zeros))
Data columns (total 2 columns):
NB      25 non-null object
ZINB    25 non-null object
dtypes: object(2)
memory usage: 641.0+ bytes


In [17]:
res = pivot.to_latex(escape=False)
res_blocks = res.replace('\midrule','').split("\n$\\lambda")
res_new = ''
print(res)
for block in res_blocks:
    res_new += block + ("\n\midrule\n$\\")
res_new = res_new[:-len("\n\midrule\n$\\")]

\begin{tabular}{llll}
\toprule
       & Model &                              NB &                            ZINB \\
Dataset & Metric &                                 &                                 \\
\midrule
Cortex & Imputation Score &      $\mathbf{2.318 \pm 0.153}$ &               $2.458 \pm 0.291$ \\
       & Log Likelihood &  $\mathbf{1276.231 \pm 18.532}$ &           $1285.789 \pm 27.041$ \\
       & PPC (Coefficient of Variation) &      $\mathbf{0.213 \pm 0.034}$ &               $0.247 \pm 0.039$ \\
       & PPC (Dropout) &                $0.087 \pm 0.04$ &      $\mathbf{0.049 \pm 0.018}$ \\
       & PPC (Ratio of zeros) &               $0.109 \pm 0.056$ &               $0.107 \pm 0.069$ \\
Hemato & Imputation Score &               $0.748 \pm 0.027$ &      $\mathbf{0.634 \pm 0.029}$ \\
       & Log Likelihood &             $914.148 \pm 7.552$ &    $\mathbf{906.609 \pm 6.603}$ \\
       & PPC (Coefficient of Variation) &               $0.214 \pm 0.106$ &      $\mathbf{0.172 

In [16]:
print(res_new)

\begin{tabular}{llll}
\toprule
       & Model &                              NB &                            ZINB \\
Dataset & Metric &                                 &                                 \\

Cortex & Imputation Score &      $\mathbf{2.318 \pm 0.153}$ &               $2.458 \pm 0.291$ \\
       & Log Likelihood &  $\mathbf{1276.231 \pm 18.532}$ &           $1285.789 \pm 27.041$ \\
       & PPC (Coefficient of Variation) &      $\mathbf{0.213 \pm 0.034}$ &               $0.247 \pm 0.039$ \\
       & PPC (Dropout) &                $0.087 \pm 0.04$ &      $\mathbf{0.049 \pm 0.018}$ \\
       & PPC (Ratio of zeros) &               $0.109 \pm 0.056$ &               $0.107 \pm 0.069$ \\
Hemato & Imputation Score &               $0.748 \pm 0.027$ &      $\mathbf{0.634 \pm 0.029}$ \\
       & Log Likelihood &             $914.148 \pm 7.552$ &    $\mathbf{906.609 \pm 6.603}$ \\
       & PPC (Coefficient of Variation) &               $0.214 \pm 0.106$ &      $\mathbf{0.172 \pm 0.08

In [25]:
# import io #Used as buffer


# def convertToLaTeX(df, alignment="c"):
#     """
#     Convert a pandas dataframe to a LaTeX tabular.
#     Prints labels in bold, does not use math mode
#     """
#     numColumns = df.shape[1]
#     numRows = df.shape[0]
#     output = io.StringIO()
#     colFormat = ("%s|%s" % (alignment, alignment * numColumns))
#     #Write header
#     output.write("\\begin{tabular}{%s}\n" % colFormat)
#     columnLabels = ["\\textbf{%s}" % label for label in df.columns]
#     output.write("& %s\\\\\\hline\n" % " & ".join(columnLabels))
#     #Write data lines
#     for i in range(numRows):
#         output.write("\\textbf{%s} & %s\\\\\n"
#                      % (df.index[i], " & ".join([str(val) for val in df.ix[i]])))
#     #Write footer
#     output.write("\\end{tabular}")
#     return output.getvalue()

# convertToLaTeX(pivot)

In [40]:
path = os.path.join(data_path, '{}.tex'.format(dataset_name))
with open(path, 'w') as tf:
     tf.write(res)

In [95]:
path

'../../ppc/scripts/log_poisson_zifa_dataset_12000_0.15_1000.0.tex'